In [1]:
!pip install deepxde

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.2/166.2 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 9.7 MB/s eta 0:00:00


In [4]:
"""Backend supported: tensorflow.compat.v1, tensorflow, pytorch, jax, paddle"""
import deepxde as dde
import numpy as np


def ode_system(x, y):
    """ODE system.
    dy1/dx = y2
    dy2/dx = -y1
    """
    # Most backends
    y1, y2 = y[:, 0:1], y[:, 1:]
    dy1_x = dde.grad.jacobian(y, x, i=0)
    dy2_x = dde.grad.jacobian(y, x, i=1)
    # Backend jax
    # y_val, y_fn = y
    # y1, y2 = y_val[:, 0:1], y_val[:, 1:]
    # dy1_x, _ = dde.grad.jacobian(y, x, i=0)
    # dy2_x, _ = dde.grad.jacobian(y, x, i=1)
    return [dy1_x - y2, dy2_x + y1]


def boundary(_, on_initial):
    return on_initial


def func(x):
    """
    y1 = sin(x)
    y2 = cos(x)
    """
    return np.hstack((np.sin(x), np.cos(x)))

#Uncomment for Complex Values
val = 1+2j
val1 = 0+1j
val2 = 1+1j
"""
val = 1.0
val1 = 0.0
val2 = 1.0
"""
geom = dde.geometry.TimeDomain(0, val)
ic1 = dde.icbc.IC(geom, lambda x: val1, boundary, component=0)
ic2 = dde.icbc.IC(geom, lambda x: val2, boundary, component=1)
data = dde.data.PDE(geom, ode_system, [ic1, ic2], 35, 2, solution=func, num_test=100)

layer_size = [1] + [50] * 3 + [2]
activation = "tanh"
initializer = "Glorot uniform"
net = dde.nn.FNN(layer_size, activation, initializer)

model = dde.Model(data, net)
model.compile("adam", lr=0.001, metrics=["l2 relative error"])
losshistory, train_state = model.train(iterations=20000)

dde.saveplot(losshistory, train_state, issave=True, isplot=True)

Compiling model...
Building feed-forward neural network...
'build' took 0.070318 s



/usr/local/lib/python3.10/dist-packages/deepxde/geometry/geometry_1d.py:132: ComplexWarning: Casting complex values to real discards the imaginary part
  return (self.diam * x + self.l).astype(config.real(np))
/usr/local/lib/python3.10/dist-packages/deepxde/geometry/geometry_1d.py:143: ComplexWarning: Casting complex values to real discards the imaginary part
  return np.array([[self.l], [self.r]]).astype(config.real(np))
/usr/local/lib/python3.10/dist-packages/numpy/core/function_base.py:175: ComplexWarning: Casting complex values to real discards the imaginary part
  return y.astype(dtype, copy=False)


TypeError: ignored